In [48]:
from mintalib import model
from mintalib import functions

from mintalib.samples import sample_prices
from mintalib.utils import format_partial
from mintalib.model import FuncIndicator


In [49]:
prices = sample_prices()
prices

,open,high,low,close,volume
date,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600
...,...,...,...,...,...
2024-09-12,222.500000,223.550003,219.820007,222.770004,37498200
2024-09-13,223.580002,224.039993,221.910004,222.500000,36766600
2024-09-16,216.539993,217.220001,213.919998,216.320007,59357400


In [50]:
help(functions.SMA)

Help on cython_function_or_method in module mintalib.core:

SMA(series, period: 'int', *, item: 'str' = None)
    Simple Moving Average
    
    Args:
        period (int) : time period, required



In [51]:
def XYZ(prices, period: int = 20, *, flag: bool = False):
    pass

format_partial(XYZ, dict(period=10, flag=False))



'XYZ(10)'

In [52]:
from mintalib.core import wrap_function, wrap_indicator

class FuncIndicator(model.Indicator):
    def __init__(self, func, params: dict, name:str):
        self.func = func
        self.params= params
        self.name = name

    def __repr__(self):
        return self.name

    def __call__(self, prices):
        return self.func(prices, **self.params)
    
    @classmethod
    def wrap(cls, func, params:dict):
        name = format_partial(func, params)
        return cls(func, params=params, name=name)
        

In [53]:
import inspect

print(inspect.getsource(format_partial))

def format_partial(func, params):
    signature = Signature.from_callable(func)
    positional = True
    arguments = []

    for k, v in params.items():
        p = signature.parameters.get(k)

        if not p or p.kind not in (
            Parameter.POSITIONAL_ONLY,
            Parameter.POSITIONAL_OR_KEYWORD,
        ):
            positional = False

        if positional:
            arguments.append(f"{v!r}")
        elif not p or v != p.default:
            arguments.append(f"{k!s}={v!r}")

    arguments = ", ".join(arguments)
    return "%s(%s)" % (func.__name__, arguments)



In [55]:
from mintalib.core import wrap_function

@wrap_indicator(functions.SMA)
def SMA(period: int = 3):
    params = locals()
    name = format_partial(SMA, params)
    return FuncIndicator(functions.EMA, params=params, name=name)

ema = SMA()
help(SMA)
ema

Help on function SMA in module __main__:

SMA(period: int = 3)
    Simple Moving Average
    
    Args:
        period (int) : time period, required



SMA(3)

In [8]:
ema(prices)

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16      0.091630
1980-12-17      0.090339
1980-12-18      0.090985
                 ...    
2024-09-12    222.234788
2024-09-13    222.367394
2024-09-16    219.343701
2024-09-17    218.066847
2024-09-18    219.378425
Length: 11033, dtype: float64

In [9]:
help(SMA)

Help on function SMA in module __main__:

SMA(period: int = 3)
    Simple Moving Average
    
    Args:
        period (int) : time period, required



In [10]:
def print_locals(period: int = 3, flag: bool = False):
    print(locals())


In [11]:
print_locals(flag=12)

{'period': 3, 'flag': 12}


In [63]:
from mintalib.core import wrap_function
from mintalib.model import FuncIndicator

@wrap_function(functions.SMA)
def SMA(period: int = 3):
    return FuncIndicator.wrap(functions.SMA, params=locals())

help(SMA)
sma = SMA()
sma

Help on function SMA in module __main__:

SMA(period: int = 3)
    Simple Moving Average
    
    Args:
        period (int) : time period, required



SMA(3)

In [64]:
sma(prices)

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16      0.093207
1980-12-17      0.089909
1980-12-18      0.089192
                 ...    
2024-09-12    221.846670
2024-09-13    222.643336
2024-09-16    220.530004
2024-09-17    218.536667
2024-09-18    217.933334
Length: 11033, dtype: float64